In [1]:
import pandas as pd
import numpy as np

# ✅ Create a dictionary manually from the table
data = {
    'Model': [
        'EfficientNet V2-S', 'EfficientNet V2-S', 'EfficientNet V2-S', 'EfficientNet V2-S',
        'ShuffleNet V2', 'ShuffleNet V2', 'ShuffleNet V2', 'ShuffleNet V2',
        'MobileNet V3', 'MobileNet V3', 'MobileNet V3', 'MobileNet V3',
        'Ensemble MESNET', 'Ensemble MESNET', 'Ensemble MESNET', 'Ensemble MESNET'
    ],
    'Metric': [
        'Accuracy', 'Precision', 'Recall', 'F1 Score',
        'Accuracy', 'Precision', 'Recall', 'F1 Score',
        'Accuracy', 'Precision', 'Recall', 'F1 Score',
        'Accuracy', 'Precision', 'Recall', 'F1 Score'
    ],
    'Seed 1': [91.87, 91.89, 91.87, 91.86, 91.79, 91.75, 91.75, 91.73, 92.45, 92.59, 92.45, 92.47, 96.84, 96.87, 96.84, 96.84],
    'Seed 2': [92.34, 92.38, 92.34, 92.33, 91.75, 91.75, 91.75, 91.73, 92.51, 92.58, 92.51, 92.49, 97.30, 97.31, 97.30, 97.30],
    'Seed 3': [92.08, 92.14, 92.08, 92.08, 91.79, 91.85, 91.79, 91.78, 92.30, 92.45, 92.30, 92.32, 96.96, 96.99, 96.96, 96.96],
    'Seed 4': [91.90, 91.95, 91.90, 91.90, 91.80, 91.83, 91.80, 91.80, 92.67, 92.75, 92.67, 92.69, 97.04, 97.06, 97.04, 97.03],
    'Seed 5': [92.23, 92.26, 92.23, 92.19, 91.57, 91.63, 91.57, 91.56, 92.87, 92.93, 92.87, 92.87, 96.78, 96.80, 96.78, 96.78]
}

# ✅ Convert to DataFrame
df = pd.DataFrame(data)

# ✅ Calculate mean and standard deviation
seed_cols = ['Seed 1', 'Seed 2', 'Seed 3', 'Seed 4', 'Seed 5']
df['Mean'] = df[seed_cols].mean(axis=1)
df['Std Dev'] = df[seed_cols].std(axis=1)

# ✅ Combine Mean ± SD in one column
df['Mean ± SD'] = df['Mean'].round(2).astype(str) + " ± " + df['Std Dev'].round(2).astype(str)

# ✅ Select clean output columns
result = df[['Model', 'Metric', 'Mean ± SD']]

# ✅ Display
print(result.to_string(index=False))


            Model    Metric    Mean ± SD
EfficientNet V2-S  Accuracy  92.08 ± 0.2
EfficientNet V2-S Precision 92.12 ± 0.21
EfficientNet V2-S    Recall  92.08 ± 0.2
EfficientNet V2-S  F1 Score  92.07 ± 0.2
    ShuffleNet V2  Accuracy  91.74 ± 0.1
    ShuffleNet V2 Precision 91.76 ± 0.09
    ShuffleNet V2    Recall 91.73 ± 0.09
    ShuffleNet V2  F1 Score 91.72 ± 0.09
     MobileNet V3  Accuracy 92.56 ± 0.22
     MobileNet V3 Precision 92.66 ± 0.18
     MobileNet V3    Recall 92.56 ± 0.22
     MobileNet V3  F1 Score 92.57 ± 0.21
  Ensemble MESNET  Accuracy  96.98 ± 0.2
  Ensemble MESNET Precision  97.01 ± 0.2
  Ensemble MESNET    Recall  96.98 ± 0.2
  Ensemble MESNET  F1 Score  96.98 ± 0.2


In [4]:
# 📊 Statistical Significance Testing (Paired t-test vs Ensemble MESNET)
# --------------------------------------------------------------------

import numpy as np
import pandas as pd
from scipy.stats import ttest_rel

# ---------------------------------------------
# Raw Data (5 Random Seeds for each model)
# ---------------------------------------------
data = {
    "EfficientNet V2-S": {
        "Accuracy": [91.87, 92.34, 92.08, 91.90, 92.23],
        "Precision": [91.89, 92.38, 92.14, 91.95, 92.26],
        "Recall": [91.87, 92.34, 92.08, 91.90, 92.23],
        "F1 Score": [91.86, 92.33, 92.08, 91.90, 92.19]
    },
    "ShuffleNet V2": {
        "Accuracy": [91.79, 91.75, 91.79, 91.80, 91.57],
        "Precision": [91.75, 91.75, 91.85, 91.83, 91.63],
        "Recall": [91.75, 91.75, 91.79, 91.80, 91.57],
        "F1 Score": [91.73, 91.73, 91.78, 91.80, 91.56]
    },
    "MobileNet V3": {
        "Accuracy": [92.45, 92.51, 92.30, 92.67, 92.87],
        "Precision": [92.59, 92.58, 92.45, 92.75, 92.93],
        "Recall": [92.45, 92.51, 92.30, 92.67, 92.87],
        "F1 Score": [92.47, 92.49, 92.32, 92.69, 92.87]
    },
    "Ensemble MESNET": {
        "Accuracy": [96.84, 97.30, 96.96, 97.04, 96.78],
        "Precision": [96.87, 97.31, 96.99, 97.06, 96.80],
        "Recall": [96.84, 97.30, 96.96, 97.04, 96.78],
        "F1 Score": [96.84, 97.30, 96.96, 96.03, 96.78]
    }
}

# ---------------------------------------------
# Helper function to format p-values as 1.37 × 10⁻⁹
# ---------------------------------------------
def format_pvalue(p):
    if p < 0.0001:
        exponent = int(np.floor(np.log10(p)))
        base = p / (10 ** exponent)
        return f"{base:.2f} × 10⁻{abs(exponent)}"
    else:
        return f"{p:.4f}"

# ---------------------------------------------
# Perform Paired t-tests vs Ensemble MESNET
# ---------------------------------------------
ensemble = data["Ensemble MESNET"]
results = []

for model in ["EfficientNet V2-S", "ShuffleNet V2", "MobileNet V3"]:
    row = {"Comparison": model}
    for metric in ["Accuracy", "Precision", "Recall", "F1 Score"]:
        t_stat, p_val = ttest_rel(data[model][metric], ensemble[metric])
        significance = "✔️ Significant (p<0.05)" if p_val < 0.05 else "❌ Not Significant"
        row[f"{metric} p-value"] = format_pvalue(p_val)
        row[f"{metric} Significance"] = significance
    results.append(row)

# ---------------------------------------------
# Create a formatted results table
# ---------------------------------------------
df_results = pd.DataFrame(results)
pd.set_option("display.max_columns", None)
pd.set_option("display.colheader_justify", "center")

print("Statistical Significance (Paired t-test vs Ensemble MESNET)\n")
display(df_results)

# ---------------------------------------------
# Optional: Save to CSV file
# ---------------------------------------------
df_results.to_csv("t_test_results_vs_MESNET.csv", index=False)
print("\n✅ Results saved to 't_test_results_vs_MESNET.csv'")


Statistical Significance (Paired t-test vs Ensemble MESNET)



,Comparison,Accuracy p-value,Accuracy Significance,Precision p-value,Precision Significance,Recall p-value,Recall Significance,F1 Score p-value,F1 Score Significance
0,EfficientNet V2-S,9.27 × 10⁻7,✔️ Significant (p<0.05),8.71 × 10⁻7,✔️ Significant (p<0.05),9.27 × 10⁻7,✔️ Significant (p<0.05),8.18 × 10⁻6,✔️ Significant (p<0.05)
1,ShuffleNet V2,3.77 × 10⁻7,✔️ Significant (p<0.05),3.44 × 10⁻7,✔️ Significant (p<0.05),3.01 × 10⁻7,✔️ Significant (p<0.05),2.22 × 10⁻5,✔️ Significant (p<0.05)
2,MobileNet V3,8.15 × 10⁻6,✔️ Significant (p<0.05),7.27 × 10⁻6,✔️ Significant (p<0.05),8.15 × 10⁻6,✔️ Significant (p<0.05),9.31 × 10⁻5,✔️ Significant (p<0.05)



✅ Results saved to 't_test_results_vs_MESNET.csv'
